In [ ]:
import pandas as pd
import numpy as np
import geopandas  as gpd
import altair as alt
from datetime import timedelta
from datetime import datetime as dt
import datetime
import json
from altair import datum

alt.data_transformers.disable_max_rows()

In [ ]:

#highlighting important NPIs in WA
data = {'date': [ "2020-03-23", "2020-06-01", "2020-11-18", "2021-02-14"], 'event':[ "Stay at home", "Stay at home lifted", "Closing restaurants", "Reopening restaurants"]}

npidf = pd.DataFrame(data)
npidf.date = pd.to_datetime(npidf.date)

rule = alt.Chart(npidf).mark_rule(
    color="black",
    strokeWidth=2, 
    opacity = 0.3
).encode(
    alt.X('date:T', axis=alt.Axis(title=None))
).properties(
    width=800,
    height=300
)

text = alt.Chart(npidf).mark_text(
    align='left',
    baseline='middle',
    dx=2,
    dy=-135,
    size=13
).encode(
    alt.X('date:T',axis=alt.Axis(title=None)),
    text='event',
    color=alt.value('#000000')
).properties(
    width=800,
    height=300
)

In [ ]:
ne_df = pd.read_csv("../data-files/ne_df.csv")
jumps_df = pd.read_csv("../data-files/migration_jumps_df.csv")
root_df = pd.read_csv("../data-files/root_states_df.csv")
persistance_df = pd.read_csv("../data-files/persistance_df.csv")
predictor_df = pd.read_csv("../data-files/predictor_coefficients.csv")

In [ ]:
jumps_df.rename(columns={'migration_direction': 'Migration Direction'}, inplace=True)
jumps_df['Migration Direction'][jumps_df['Migration Direction'] == "North_King_County-to-South_King_County"] = "North to South King County"
jumps_df["Migration Direction"][jumps_df['Migration Direction'] == "South_King_County-to-North_King_County"] = "South to North King County"

root_df.rename(columns={'region': 'Region'}, inplace=True)


In [ ]:
persistance_df.rename(columns={'child_host': 'Region'}, inplace=True)
persistance_df.Region[persistance_df.Region == "North_King_County"] = "North King County"
persistance_df.Region[persistance_df.Region == "South_King_County"] = "South King County"

In [ ]:
line = alt.Chart(ne_df).mark_area(interpolate='monotone').encode(
    alt.X('date:T', axis=alt.Axis(title="Date", grid=False, format="%B %Y")),
    alt.Y('lower_hpd_log_50',axis=alt.Axis(title="Effective Population Size", grid=False)),#,scale=alt.Scale(domain=(0, 13))),
    alt.Y2('upper_hpd_log_50' ),
    color=alt.Color('deme:N')
).properties(
    width=1000,
    height=300
).transform_filter(
    (datum.upper_hpd_log_50 < 95)
)

band = alt.Chart(ne_df).mark_area(
    opacity=0.3, interpolate='monotone'
).encode(
    alt.X('date:T', axis=alt.Axis(title="Date", grid=False)),
    alt.Y('lower_hpd_log_95'),#axis=None),#, scale=alt.Scale(domain=(0, 13))),
    alt.Y2('upper_hpd_log_95'),
    color=alt.Color('deme:N', legend=None)
).properties(
    width=1000,
    height=300
).transform_filter(
    (datum.upper_hpd_log_95 < 95)
)

band + line

In [ ]:
ne = (band + line + text + rule)

In [ ]:
ne

In [ ]:
ne_pred = predictor_df.iloc[1:6]
mig_pred = predictor_df.iloc[-2:]

In [ ]:
ne_pred = ne_pred.replace(["Ne_hosp_orig", "Ne_hosp_1wk_fwd", "Ne_hosp_2wk_fwd", "percent_immunity", "cases"], ["Hospializations (no lag)","Hospializations (1wk lag)", "Hospializations (2wk lag)", "Percent Immunity", "Cases" ])
mig_pred

In [ ]:
mig_pred = mig_pred.replace(["NPI_dates", "safegraph_between_total_mvmt"], ["NPI Dates", "Safegraph Mobility"])

In [ ]:
# error_bars = alt.Chart(mig_short).mark_errorbar(extent='ci').encode(
#   x=alt.X('mig_per_bl:Q', scale=alt.Scale(zero=False)),
#   y=alt.Y('migration_direction:N')
# )

points = alt.Chart(ne_pred).mark_point(filled=True, color='black', width = 5).encode(
  y=alt.Y('mean_Ne_log:Q',axis = alt.Axis(title = "Coefficient", grid =False,) ),
  x=alt.X('deme', title = "Predictor",   axis=alt.Axis(labelLimit = 0, labelAngle=-35)),
).transform_filter(datum.predictor != "Clock" or datum.predictor != "air").properties(
    width=650,
    height=200
)

hpd = alt.Chart(ne_pred).mark_rule(strokeWidth = 2).encode(
    y2=alt.Y2("upper_hpd_log_95"),
    y = alt.Y("lower_hpd_log_95"),
    x=alt.X("deme")
).transform_filter(datum.predictor != "Clock").properties(
    width=850,
    height=300
)

one_line = alt.Chart(pd.DataFrame({'y': [0.0]})).mark_rule(strokeDash=[1,1]).encode(y='y').properties(
    width=850,
    height=300
)

predict_coeff_ne = (points + hpd + one_line).properties(title='B')
# predict_coeff_ne.configure_view(
#     strokeWidth=0
# ).configure_title(
#     anchor='start', fontSize= 35
# ).configure_axis(
#     labelFontSize=60,
#     titleFontSize=60
# ).configure_legend(
#     labelFontSize = 60)

predict_coeff_ne


In [ ]:
# error_bars = alt.Chart(mig_short).mark_errorbar(extent='ci').encode(
#   x=alt.X('mig_per_bl:Q', scale=alt.Scale(zero=False)),
#   y=alt.Y('migration_direction:N')
# )

points = alt.Chart(mig_pred).mark_point(filled=True, color='black').encode(
  y=alt.Y('mean_Ne_log:Q',axis = alt.Axis(title = "Coefficient", grid =False, labelFontSize= 18,titleFontSize= 16) ),
  x=alt.X('deme', title = "Predictor",  axis=alt.Axis(labelFontSize= 16,titleFontSize= 16, labelAngle=-35)),
).transform_filter(datum.predictor != "Clock" ).properties(
    width=250,
    height=200
)

hpd = alt.Chart(mig_pred).mark_rule().encode(
    y2=alt.Y2("upper_hpd_log_95"),
    y = alt.Y("lower_hpd_log_95"),
    x=alt.X("deme")
).transform_filter(datum.predictor != "Clock").properties(
    width=300,
    height=150
)

one_line = alt.Chart(pd.DataFrame({'y': [0.0]})).mark_rule(strokeDash=[1,1]).encode(y='y').properties(
    width=300,
    height=150
)

predict_coeff_mig = (points + hpd + one_line ).properties(title='C')
predict_coeff_mig.configure_view(
    strokeWidth=0
).configure_title(
    anchor='start', fontSize= 25
).configure_axis(
    labelFontSize=24,
    titleFontSize=24
).configure_legend(
    labelFontSize = 60)




In [ ]:
# Add labels to each chart
ne = ne.properties(title='A')


# Concatenate labeled charts B and C horizontally
predictors  = alt.hconcat(predict_coeff_ne , predict_coeff_mig).resolve_scale(color = "independent")

# Concatenate labeled charts AB and CD vertically
chart_abcd_labeled = alt.vconcat(ne, predictors, spacing=50)#.resolve_scale(x='shared', color ='shared')

chart_abcd_labeled.configure_view(
    strokeWidth=0
).configure_title(
    anchor='start', fontSize= 30
).configure_axis(
    labelFontSize=14,
    titleFontSize=14
).configure_legend(
    labelFontSize = 14)


In [ ]:
error_bars_pr = alt.Chart(persistance_df).mark_errorbar(extent='ci').encode(
  x=alt.X('year-month:T', axis=alt.Axis( grid=False, format="%B %Y")),
  y=alt.Y('persistence_time:Q',axis=alt.Axis(title="Length of Local Transmission (in days)", grid=False)), 
  color = alt.Color("Region:N")
).properties(
    width=1000,
    height=300
)

points_pr = alt.Chart(persistance_df).mark_point(filled=True,  opacity = 1, size = 100).encode(
  x=alt.X('year-month:T'),
  y=alt.Y('persistence_time:Q', aggregate='mean'),
    color = alt.Color("Region:N",  legend=alt.Legend(offset = -220, labelFontSize = 12, titleFontSize = 12))
).properties(
    width=1000,
    height=300
)

lineplot_pr =  alt.Chart(persistance_df).mark_line(interpolate='monotone', opacity = 0.35).encode(
    x=alt.X('year-month:T'),
    y=alt.Y('mean(persistence_time)'),
    color=alt.Color('Region:N')).properties(
    width=1000,
    height=300
)


persistence = error_bars_pr + points_pr +lineplot_pr + rule + text 
persistence

In [ ]:
error_bars = alt.Chart(persistance_df).mark_errorbar(extent='ci').encode(
  y=alt.Y('persistence_time:Q', scale=alt.Scale(zero=False), axis=alt.Axis(title="Average Length of Local Transmission (in days)", grid = False,orient = "right")),
  x=alt.X('Region:N', axis=alt.Axis(labels=False)), color = alt.Color("Region:N", legend = None)
)

points = alt.Chart(persistance_df).mark_point(filled=True, color='black', size = 60).encode(
  y=alt.Y('persistence_time:Q', aggregate='mean'),
  x=alt.X('Region:N', axis=alt.Axis(title=None)),  color = alt.Color("Region:N", legend = None)
)

ave_persist = error_bars + points
#ave.save("jumps_error.png")
ave_persist

In [ ]:
persist = persistence | ave_persist
persist

In [ ]:
lineplot3 =  alt.Chart(jumps_df, width = 750).mark_line(interpolate='monotone', clip = True).encode(
    x=alt.X('year-month:T',axis=alt.Axis( grid=False)),
    y=alt.Y('mean(mig_per_bl)',axis=alt.Axis(title="Number of Migration Events (Normalized)", grid=False)),
    color=alt.Color('Migration Direction:N', legend=alt.Legend(offset = -160, labelFontSize = 12, titleFontSize = 12), scale=alt.Scale(scheme='accent'))).properties(
    width=1000,
    height=300
).transform_filter(
    (datum.mig_per_bl < 150)
)


band3 = alt.Chart(jumps_df).mark_errorband(extent='ci', interpolate='monotone', clip = True).encode(
    x=alt.X('year-month:T'),
    y=alt.Y('mig_per_bl',axis=alt.Axis(title="", grid=False)), 
    color =alt.Color('Migration Direction:N', scale=alt.Scale(scheme='accent'))).properties(
    width=1000,
    height=300
).transform_filter(
    (datum.mig_per_bl < 150)
)

jumps_per_bl = lineplot3 + band3 + rule + text 
jumps_per_bl

In [ ]:
error_bars = alt.Chart(jumps_df).mark_errorbar(extent='ci').encode(
  y=alt.Y('mig_per_bl:Q', scale=alt.Scale(zero=False), axis=alt.Axis(title="Average Number of Migration Events",grid = False, orient = "right")),
  x=alt.X('Migration Direction:N', axis=alt.Axis(labels=False)), color = alt.Color("Migration Direction:N", legend = None, scale=alt.Scale(domain = ['North to South King County', 'South to North King County'], range = ['#9461bd',"#2ca02c"]))
)

points = alt.Chart(jumps_df).mark_point(filled=True, color='black', size = 100).encode(
  y=alt.Y('mig_per_bl:Q', aggregate='mean'),
  x=alt.X('Migration Direction:N', axis=alt.Axis(title=None)),  color = alt.Color("Migration Direction:N", legend = None, scale=alt.Scale(domain = ['North to South King County', 'South to North King County'], range = ['#9461bd',"#2ca02c"]))
)

ave_jumps = error_bars + points
#ave.save("jumps_error.png")
ave_jumps

In [ ]:
error_bars = alt.Chart(jumps_df).mark_errorbar(extent='ci', clip = True).encode(
  x=alt.X('year-month:T', axis=alt.Axis( grid=False, format="%B %Y")),
  y=alt.Y('mig_per_bl:Q',axis=alt.Axis(title="Number of Migration Events (Normalized)", grid=False), scale = alt.Scale(domain = (0,100))), color = alt.Color("Migration Direction:N", scale=alt.Scale(domain = ['North to South King County', 'South to North King County'], range = ['#9461bd',"#2ca02c"]))
).properties(
    width=1000,
    height=300
)

points = alt.Chart(jumps_df).mark_point(filled=True,  opacity = 1, size = 100, clip = True).encode(
  x=alt.X('year-month:T'),
  y=alt.Y('mig_per_bl:Q', aggregate='mean', scale = alt.Scale(domain = (0,100))),
    color = alt.Color("Migration Direction:N",  legend=alt.Legend(offset = -275, labelFontSize = 12, titleFontSize = 12),scale=alt.Scale(domain = ['North to South King County', 'South to North King County'], range = ['#9461bd',"#2ca02c"]))
).properties(
    width=1000,
    height=300
)

lineplot4 =  alt.Chart(jumps_df).mark_line(interpolate='monotone', opacity = 0.35, clip = True).encode(
    x=alt.X('year-month:T'),
    y=alt.Y('mean(mig_per_bl)', scale = alt.Scale(domain = (0,100))),
    color=alt.Color('Migration Direction:N', scale=alt.Scale(domain = ['North to South King County', 'South to North King County'], range = ['#9461bd',"#2ca02c"]))).properties(
    width=1000,
    height=300
)


jumps_per_bl = error_bars + points +lineplot4 + rule + text 
jumps_per_bl

In [ ]:
jumps = jumps_per_bl |ave_jumps
jumps

In [ ]:
root_states = alt.Chart(root_df).mark_bar(size=30).encode(
    alt.X('year-month:T', axis=alt.Axis(title="", grid=False, format="%B %Y")), 
    alt.Color('Region:N', legend = None),
    alt.Y("mean(mig_per_bl)", stack="normalize", title='Cluster Root States',axis=alt.Axis(offset=10 ,format='%'))).properties(
    width=1000,
    height=300
)
root_states.configure_axis(
    labelFontSize=14,
    titleFontSize=14
)

In [ ]:
error_bars = alt.Chart(root_df).mark_errorbar(extent='ci').encode(
  y=alt.Y('mig_per_bl:Q', scale=alt.Scale(zero=False), axis=alt.Axis(title="Average Cluster Root States", grid = False, orient = "right")),
  x=alt.X('Region:N', axis=alt.Axis(labels=False)), color = alt.Color("Region:N", legend = None)
)

points = alt.Chart(root_df).mark_point(filled=True, color='black', size = 100).encode(
  y=alt.Y('mig_per_bl:Q', aggregate='mean'),
  x=alt.X('Region:N', axis=alt.Axis(title=None)),  color = alt.Color("Region:N", legend = None)
)

ave_root = error_bars + points
#ave.save("jumps_error.png")
ave_root

In [ ]:
root = root_states | ave_root
root

In [ ]:
ne
#ne.save("figure_3_ne.png")

In [ ]:
figure4 = (persist & root & jumps).resolve_scale(color = "independent").configure_axis(
    labelFontSize=12,
    titleFontSize=14
)
figure4

In [ ]:
figure4.save("../figures/figure4.html")